In [37]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
hrb=LancasterStemmer()

In [38]:
import numpy
import tflearn
import tensorflow
import random
import json

In [39]:
with open("intents.json") as file:
    data = json.load(file)

In [40]:
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me HRBot.', 'HRBot'], 'context_set': ''}, {'tag': 'Salary', 'patterns': ['What is My salary?', 'Salary', 'Please tell my salary?'], 'responses': ['$35000', '$45000', '$50000'], 'context_set': ''}, {'tag': 'Tax', 'patterns': ["How much I'm being taxed?", 'Tax'], 'responses': ["You're being Taxed at 20% anuual rate in current slab"], 'context_set': ''}, {'tag': 'Leave', 'patterns': ['How many Leaves do i have?', 'Leaves'], 'responses': ['You have 20 Annual 

In [41]:
w=[]
l=[]
d_x=[]
d_y=[]
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        ws=nltk.word_tokenize(pattern)
        w.extend(ws)
        d_x.append(ws)
        d_y.append(intent["tag"])
        if intent["tag"] not in l:
            l.append(intent["tag"])
        
        

In [42]:
w=[hrb.stem(r.lower()) for r in w if w !="?"]
w=sorted(list(set(w)))

In [43]:
l=sorted(l)

In [44]:
tr=[]
op=[]
out_empty=[0 for _ in range(len(l))]

In [45]:
for x, doc in enumerate(d_x):
    bag = []

    wrds = [hrb.stem(z.lower()) for z in doc]

    for r in w:
        if r in ws:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[l.index(d_y[x])] = 1

    tr.append(bag)
    op.append(output_row)


tr = numpy.array(tr)
op = numpy.array(op)

In [46]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(tr[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(op[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [47]:
model.fit(tr, op, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 2999  | total loss: 1.85021 | time: 0.004s
| Adam | epoch: 1000 | loss: 1.85021 - acc: 0.2625 -- iter: 16/23
Training Step: 3000  | total loss: 1.85749 | time: 0.007s
| Adam | epoch: 1000 | loss: 1.85749 - acc: 0.2791 -- iter: 23/23
--
INFO:tensorflow:E:\cbr\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [48]:
try:
    model.load("model.tflearn")
except:
    model.fit(tr, op, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from E:\cbr\model.tflearn


In [49]:
def bag_of_words(s, w):
    bag = [0 for _ in range(len(w))]

    s_words = nltk.word_tokenize(s)
    s_words = [hrb.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, r in enumerate(w):
            if r == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, w)])
        results_index = numpy.argmax(results)
        tag = l[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))


In [50]:
chat()

Start talking with the bot (type quit to stop)!
You: Leaves
Hello!
You: Complaint
Hi there, how can I help?
You: Salary
Hi there, how can I help?
You: quit
